<a href="https://colab.research.google.com/github/DiMorten/FCN_ConvLSTM_Crop_Recognition_Open_Set/blob/coords3/train_and_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
!pip install icecream
%tensorflow_version 1.x
import os
!pip install kora
from kora import drive
import time
!pip install colorama

ds_path='/content/drive/My Drive/PhD/datasets/cv_data/'

TensorFlow 1.x selected.
     |████████████████████████████████| 57 kB 4.0 MB/s 
     |████████████████████████████████| 53 kB 2.9 MB/s 


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
git_clone = True

if git_clone == True:
  os.chdir('/content')
  %rm -rf FCN_ConvLSTM_Crop_Recognition_Open_Set
  !git clone --branch coords3 https://github.com/DiMorten/FCN_ConvLSTM_Crop_Recognition_Open_Set.git

Cloning into 'FCN_ConvLSTM_Crop_Recognition_Open_Set'...
remote: Enumerating objects: 1725, done.
remote: Counting objects: 100% (1725/1725), done.
remote: Compressing objects: 100% (1223/1223), done.
remote: Total 1725 (delta 1107), reused 927 (delta 332), pack-reused 0
Receiving objects: 100% (1725/1725), 37.28 MiB | 31.06 MiB/s, done.
Resolving deltas: 100% (1107/1107), done.


In [4]:
os.chdir('/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src')
os.getcwd()
os.listdir()

['obj',
 'deb.py',
 'metrics.py',
 'model.py',
 'keras_weighted_categorical_crossentropy.py',
 'dataset.py',
 'model_best_UUnet4ConvLSTM_jun_cv_criteria_0_92',
 'model_input_mode.py',
 'generator.py',
 'densnet.py',
 'parameters',
 'monitor.py',
 'densnet_timedistributed.py',
 'patch_extractor.py',
 'analysis',
 '__init__.py',
 'main.py']

In [5]:
from colorama import init
init()
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, Conv2DTranspose
# from keras.callbacks import ModelCheckpoint , EarlyStopping
from keras.optimizers import Adam,Adagrad 
from keras.models import Model
from keras import backend as K
import keras

import numpy as np
from sklearn.utils import shuffle
import cv2
import argparse
import tensorflow as tf

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import metrics
import sys
import glob

from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,classification_report
# Local
from densnet import DenseNetFCN
from densnet_timedistributed import DenseNetFCNTimeDistributed

#from metrics import fmeasure,categorical_accuracy
import deb
from keras_weighted_categorical_crossentropy import weighted_categorical_crossentropy, sparse_accuracy_ignoring_last_label, weighted_categorical_crossentropy_ignoring_last_label, categorical_focal_ignoring_last_label, weighted_categorical_focal_ignoring_last_label
from keras.models import load_model
from keras.layers import ConvLSTM2D, UpSampling2D, multiply
from keras.utils.vis_utils import plot_model
from keras.regularizers import l1,l2
import time
import pickle
#from keras_self_attention import SeqSelfAttention
import pdb
import pathlib
from pathlib import Path, PureWindowsPath
from keras.layers import Conv3DTranspose, Conv3D

from keras.callbacks import EarlyStopping
import tensorflow as tf
from collections import Counter


#from datagenerator import DataGenerator
from generator import DataGenerator, DataGeneratorWithCoords, DataGeneratorWithCoordsPatches

import matplotlib.pyplot as plt
sys.path.append('../../../dataset/dataset/patches_extract_script/')
from dataSource import DataSource, SARSource, OpticalSource, Dataset, LEM, LEM2, CampoVerde, OpticalSourceWithClouds, Humidity
from model_input_mode import MIMFixed, MIMVarLabel, MIMVarSeqLabel, MIMVarLabel_PaddedSeq, MIMFixedLabelAllLabels, MIMFixed_PaddedSeq
from parameters.parameters_reader import ParamsTrain

from icecream import ic
from monitor import Monitor, MonitorNPY, MonitorGenerator, MonitorNPYAndGenerator
import natsort
from model import NetModel, ModelFit, ModelLoadGenerator, ModelLoadGeneratorDebug, ModelLoadGeneratorWithCoords, ModelLoadEachBatch
from dataset import Dataset, DatasetWithCoords

from patch_extractor import PatchExtractor
ic.configureOutput(includeContext=False)
np.random.seed(2021)
tf.set_random_seed(2021)

from main import TrainTest

Using TensorFlow backend.


['parameters_openset_specifyunknownclasses.json', 'parameters_openset_lessclass8.json', 'no_mode.json', 'twokkc_parameters_closedset_groupclasses.json', 'twokkc_parameters_openset.json', 'allkkc_parameters_openset.json', 'cv', 'parameters_closedset_groupclasses_lessclass8.json', 'save_nonaugmented_train_patches_lessclass8.json', 'parameters_reader.py', 'parameters_openset.json', '__pycache__', 'parameters_closedset_groupclasses.json', 'twokkc_save_nonaugmented_train_patches.json', 'save_nonaugmented_train_patches_unknownclasses.json', 'allkkc_save_nonaugmented_train_patches.json', 'save_nonaugmented_train_patches.json', '__init__.py']


ic| self.seq_date: 'jun'


self.known_classes [1, 2, 6, 8]
[@debug] paramsTrain.seq_mode = fixed
[@debug] paramsTrain.mim = <model_input_mode.MIMFixed_PaddedSeq object at 0x7fbdc2419a50>


In [6]:
!nvidia-smi

Tue Jul 20 19:22:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download images into proper folder

In [8]:


paramsTrain = ParamsTrain('parameters/')
paramsTrain.mim = MIMFixed_PaddedSeq()

paramsTrain.dataSource = SARSource()

trainTest = TrainTest(paramsTrain)

patchExtractor = PatchExtractor(paramsTrain, trainTest.ds)	


['parameters_openset_specifyunknownclasses.json', 'parameters_openset_lessclass8.json', 'no_mode.json', 'twokkc_parameters_closedset_groupclasses.json', 'twokkc_parameters_openset.json', 'allkkc_parameters_openset.json', 'cv', 'parameters_closedset_groupclasses_lessclass8.json', 'save_nonaugmented_train_patches_lessclass8.json', 'parameters_reader.py', 'parameters_openset.json', '__pycache__', 'parameters_closedset_groupclasses.json', 'twokkc_save_nonaugmented_train_patches.json', 'save_nonaugmented_train_patches_unknownclasses.json', 'allkkc_save_nonaugmented_train_patches.json', 'save_nonaugmented_train_patches.json', '__init__.py']


ic| self.seq_date: 'jun'


self.known_classes [1, 2, 6, 8]
[@debug] self.ds = <dataSource.CampoVerde object at 0x7fbdbb3b9a90>
20151029
20151110
20151122
20151204
20151216
20160121
20160214
20160309
20160321
20160508
20160520
20160613
dotys_sin_cos.shape (12, 2)
[302, 314, 326, 338, 350, 21, 45, 69, 81, 129, 141, 165]
[[0.05084 0.7197 ]
 [0.1053  0.807  ]
 [0.1764  0.8813 ]
 [0.2612  0.9395 ]
 [0.3562  0.979  ]
 [0.6685  0.9707 ]
 [0.843   0.8643 ]
 [0.96    0.6963 ]
 [0.99    0.598  ]
 [0.905   0.2068 ]
 [0.8364  0.1301 ]
 [0.66    0.02637]]


ic| self.dataSource: <dataSource.SARSource object at 0x7fbdbb3b9e10>
ic| self.conf['path']/self.label_folder/"/": PosixPath('/')
ic| self.conf["in_npy_path"]: PosixPath('../../../dataset/dataset/cv_data/in_sar')
ic| self.conf["train"]["mask"]["dir"]: PosixPath('../../../dataset/dataset/cv_data/TrainTestMask.tif')
ic| os.getcwd(): '/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src'


## Download or load sequence of images


In [ ]:
if paramsTrain.getFullIms == True:
  patchExtractor.getFullIms()	
else:
  patchExtractor.fullImsLoad()


## Extract coords of image patches

In [ ]:

if paramsTrain.coordsExtract == True:
  patchExtractor.extract()

del patchExtractor


## Train and evaluate


In [ ]:

model_name_id = 'model_best_' + paramsTrain.model_type + '_' + \
    paramsTrain.seq_date + '_' + paramsTrain.dataset + '_' + \
    paramsTrain.model_name + '.h5'

if paramsTrain.train == True:
  trainTest.trainAndEvaluate(model_name_id)